## Logistička regresija

Logistička regresija predstavlja algoritam za binarnu klasifikaciju zasnovan na statistici. Iako je veoma jednostavan i nema veliku upotrebu danas, algoritam logističk regresije uvodi neke ideje koje su nam bitne za naredna predavanja. 

### Sigmoid funkcija

Spominjali smo prije da regresija podrazumijeva predviđanje realne vrijednosti. Isto tako i u logističkoj regresiji prvo predviđamo realnu vrijednost upotrebom formule za lineranu regresiju:

$$
Y = w * X + b
$$

Pošto sada radimo sa klasifikacijom potrebno ja da datu realnu vrijednost na neki način pretvorimo u klasu. Za pretvaranje realne vrijednosti u klasu koristimo sigmoid funkciju. Sigmodi funkcija je definisana sljedećom formulom:
$$
\sigma(x) = \frac{1}{1 + e ^{-x}}
$$

Grafik funkcije se može vidjeti na prezentaciji. Ono što je bitno je da funkcija sigmoid sabija ulaz u segment $[0, 1]$, tako da negativne realne vrijednosti teže 0 dok pozitivne vrijednosti teže 1. Sa aspekta klasifikacije izlaz funkcije sigmoid posmatramo kao vjerovatnoću da da izlaz pripada klasi 1, odnosno ako je izlaz sigmoid funkcije ispod 0.5 ulaz ćemo klasifikovati u klasu 0, a u suprotnom u klasu 1. Koristeći sigmoid funkciju možemo reći da je model logističke regresije zasnovan na sljedećoj formuli:
$$
    P(y=1 | x) = \sigma(w*x + b)
$$

### Funkcija greške

Drugi bitan aspekt na koji moramo da se osvrnemo je funkcija greške. Kod standardne linearne regresije minimizovali smo kvadrat razlike. Međutim u slučaju logističke regresije naš cilj nije minimizacija razlika predviđene realne vrijednosti i očekivane, već adekvatna klasifikacija ulaza. Osnovna ideja je da za ulaze koji pripadaju klasi 1, želimo da maksimizujemo izlaz sigmoid funkcije, odnosno da maksimizujemo vjerovatnoću koju naš model predviđa da ulaz pripada klasi 1. Dati metod se zove Maximum Likelihood Estimation (MLE) i zasnovan je na sljedećoj formuli.

$$
L(w, b) =  \prod{[P(y=1 | x)*y + (1 - P(y=1 | x))*(1-y)]}
$$

Možemo primijetiti da u slučaju kad je stvarna klasa 1, $y = 1$, posmatra se samo prvi dio formule, odnosno želimo da izlaz sigmoid funkcije bude što veći. U durgom slučaju kad je klasa 0, $y = 0$, posmatramo samo drugi član formule, odnosno želimo da nam izlaz sigmoid funkcije bude što manji. MLE možemo intuitivno posmatrati kao mjeru koliko su naše vjerovatnoće blizu stvarnim klasama. Često se zbog numeričke stabilnosti i lakšeg računanja koristi logartiam MLE formule odnosno:
$$
    LL(w, b) = \sum{[y * log(P(y=1 | x)) + (1-y) * log(1 - P(y=1 | x))]}
$$

Rekli smo da je ideja da želimo da maksimizujemo vjerovatnoću, ali u slučaju algoritama mašinskog učenja osnovni koncept je minimizacija greške. Zbog toka umjesto maksimizacije upotrebom prethodne formule $LL(w, b)$, vršimo minimizaciju upotrebom $-LL(w, b)$.

### Gradient descent

Nakon što smo izračunali grešku potrebno je da promijenimo naše težine w i b, kako bi se greška smanjila. S obizirom da znamo funkciju greške u zavisnosti od w i b, možemo potencijalno kao u slučaju linearne regresije da nađemo prvi izvod funkcije po w i b, pa da pronađemo njegove 0 kako bismo našli globalni minimum. Dati će dati najbolje rezultate međutim u praksi često nije izvodljiv zbog veoma kompleksnih zavisnosti i velikog broja parametara. Pored toga ne postoje programski okviri koji podržavaju tačno rješavanje datog problema, što bi značilo da za svaki problem moramo prvo ručno da raspišemo račun na osnovu kog bi kucali kod. Kako bismo rješili da ti problem u mašinskom učenju koristi se tehnika gradijentog spusta, koja samo zahtijeva da znamo koja je formula za prvi izvod neke funkcije ali ne uključuje računanje njegovih nula (što predstavlja veći problem). Osnovna ideja gradijetnog spusta je zasnovana na kretanju u suprtonom pravu od gradijenta funkcija. Uzmimo u obzir da prvi izvod funkcije definiše kada je ona rastuća ili opadajuća. Ako izračunarmo prvi izvod funkcije greške za neku kombinaciju parametara w i b, mi bismo željeli da se krećemo u pravcu suprotnom od rasta funkcije, zato što želimo da minimizujemo grešku. Želimo da mijenjamo naše parametre tako da se funkcija kreće u pravcu opadanja.

Koraci za gradijentni spust:
1. Inicijalizujemo model sa nasumičnim parametrima w i b
2. Izračunamo vrijednost gradijenta funkcije greške za date parametre
3. Promijenimo parametre tako da se krećemo u pravcu suprotnom od gradijenta
4. Ponavljamo korake 2 i 3 određeni broj iteracija

Funkcija promejene parametara $w$ i $b$ je definisana na sljedeći način:
$$
    w = w - \alpha * \frac{\partial LL(w, b)}{\partial w}
    b = b - \alpha * \frac{\partial LL(w, b)}{\partial b}
$$
Faktor $\alpha$ se naziva faktorom obučavanja i definiše koliko brzo ćemo mijenjati vrijednost $w$ i $b$ u svakoj iteraciji. Ukoliko je faktor obučavanja previše malen promjene će biti jako spore pa samim tim i konvergencija kao minimumu će zahtijevati veliki broj iteracija. Takođe, sa druge strane ukoliko je $\alpha$ preveliko možemo imati slučaj da stalno preskačemo minimum zbog prevelikih promjena (pogledati prezentaciju).

Veoma je bitno napomenuti da tehnikom gradijentnog spusta nikad nećemo dobiti idealne rezultate i da nikad nećemo pronaći pravi globalni minimum. Ipak nakon dovoljno iteracija sa adekvatnim faktorom obučavanja možemo prići dovoljno blizu minimumu, tako da naš model ima prihvatljive performanse. 

#### Zadatak za vježbu

Izvesti formule za $\frac{\partial LL(w, b)}{\partial w}$ i $\frac{\partial LL(w, b)}{\partial b}$.

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.datasets import load_breast_cancer


breast_cancer = load_breast_cancer()
X = breast_cancer.data
y = breast_cancer.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [5]:
import numpy as np
from sklearn.base import BaseEstimator, ClassifierMixin


def sigmoid(z):
    return 1 / (1 + np.exp(-z))


class LogisticRegressionCustom(BaseEstimator, ClassifierMixin):
    """
    Parametri
    ---------
    learning_rate : float, default=0.01
        Veličina koraka prilikom gradijentnog spusta.
    num_iterations : int, default=1000
        Broj iteracija za optimizaciju.
    threshold : float, default=0.5
        Prag za klasifikaciju (pretvaranje verovatnoće u labelu).
    verbose : bool, default=False
        Ako je True, ispisuje se gubitak svakih 100 iteracija.
    """

    def __init__(self, learning_rate=0.01, num_iterations=1000, threshold=0.5, verbose=False):
        self.learning_rate = learning_rate
        self.num_iterations = num_iterations
        self.threshold = threshold
        self.verbose = verbose
        # Atributi koji se postavljaju tokom fitovanja
        self.w_ = None
        self.b_ = None
        self.loss_history_ = []

    def fit(self, X, y):
        # Pretvaranje ulaza u numpy niz i reshaping ciljne promenljive
        X = np.asarray(X)
        y = np.asarray(y).reshape(-1, 1)
        n_samples, n_features = X.shape

        # Inicijalizacija parametara
        self.w_ = np.zeros((n_features, 1))
        self.b_ = 0

        # Gradijentni spust
        for i in range(self.num_iterations):
            z = X.dot(self.w_) + self.b_
            a = sigmoid(z)

            # Računanje gradijenata
            dw = (1 / n_samples) * X.T.dot(a - y)
            db = (1 / n_samples) * np.sum(a - y)

            # Ažuriranje parametara
            self.w_ -= self.learning_rate * dw
            self.b_ -= self.learning_rate * db

            # Praćenje gubitka
            if self.verbose and i % 100 == 0:
                loss = - (1 / n_samples) * np.sum(y * np.log(a) + (1 - y) * np.log(1 - a))
                self.loss_history_.append(loss)
                print(f"Gubitak nakon iteracije {i}: {loss}")

        return self

    def predict_proba(self, X):
        # Izračunava verovatnoće za klase 0 i 1
        X = np.asarray(X)
        z = X.dot(self.w_) + self.b_
        probs = sigmoid(z)
        return np.hstack([1 - probs, probs])

    def predict(self, X):
        # Pretvara verovatnoću klase 1 u binarnu predikciju
        probs = self.predict_proba(X)[:, 1]
        return (probs >= self.threshold).astype(int)

In [6]:
model = LogisticRegressionCustom(learning_rate=0.01, num_iterations=1000, verbose=True)
model.fit(X_train, y_train)
preds = model.predict(X_test)
print(classification_report(y_test, preds))

Gubitak nakon iteracije 0: 0.6931471805599452
Gubitak nakon iteracije 100: nan
Gubitak nakon iteracije 200: nan
Gubitak nakon iteracije 300: nan
Gubitak nakon iteracije 400: nan
Gubitak nakon iteracije 500: nan
Gubitak nakon iteracije 600: nan
Gubitak nakon iteracije 700: nan
Gubitak nakon iteracije 800: nan
Gubitak nakon iteracije 900: nan
              precision    recall  f1-score   support

           0       0.86      0.97      0.91        63
           1       0.98      0.91      0.94       108

    accuracy                           0.93       171
   macro avg       0.92      0.94      0.93       171
weighted avg       0.94      0.93      0.93       171



/tmp/ipykernel_10151/1357729101.py:6: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
/tmp/ipykernel_10151/1357729101.py:58: RuntimeWarning: divide by zero encountered in log
  loss = - (1 / n_samples) * np.sum(y * np.log(a) + (1 - y) * np.log(1 - a))
/tmp/ipykernel_10151/1357729101.py:58: RuntimeWarning: invalid value encountered in multiply
  loss = - (1 / n_samples) * np.sum(y * np.log(a) + (1 - y) * np.log(1 - a))


## LightGBM - staro gradivo (nije obavezno, zanimljivo koga zanima više)

LigthGBM je Microsoft-ov algoritam za rad sa tabularnim (podaci koji se mogu predstaviti u vidu tabele) podacima zasnovan na stablima odlučivanja. Uvodi brojne optimizacije nad standardnim stablima odlučivanja što mu daje veliku efikasnot i stabilnost, te omogućava da dostigne veoma dobre performanse na velikom broju problema. Zasnovan je na ideji Boosting-a o kojoj smo pričali na prošlom času, odnosno trenira se veći broj weak learner-a čiji je cilj da isprave greške prethodnika. Da bismo instalirali LightGBM u pythonu, koristimo komandu:
```bash
    pip install lightgbm
```

### Gradient Boosting i GOSS

Za računanje splitova i optimizaciju stabla LightGBM koristi varijantu Boosting algoritma zasnovanu na gradient descent-u (GBDT). Zbog veće računske kompleksnosti nećemo ulaziti u tačan račun, ali zainteresovani mogu pročitati originalni rad ako žele: https://proceedings.neurips.cc/paper_files/paper/2017/file/6449f44a102fde848669bdd9eb6b76fa-Paper.pdf. Dodatna optimizacija koju LightGBM uvodi prilikom računanja gradienta je fokusiranje na ulaze koji imaju visoku grešku. Data tehnika se naziva Gradient-based One-Side Sampling (GOSS) i podrazumijeva da se tokom računanja koriste samo ulazi sa visokom greškom, dok se ostali ignorišu. Data optimimizacija značajno ubrzava rad algoritma i omogućava mu da ispuni onaj Boosting aspekat, odnosno fokus na greške prethodnog modela.

### Leaf wise vs Level wise growth

Druga optimizacija koju LigthGBM uvodi je Leaf wise growth. Većina standardnih algoritama koji rade sa stablima odlučivanja koriste Level wise growth. Level wise growth podrazumijeva da na pri svakoj iteraciji prođemo kroz sve čvorove na trenutnom nivou i nađemo novi uslov grananja za svaki od njih. Takav pristup može značajno da poveća šanse za overfitting (da li ima smisla da radimo split na već homogenom podskupu), a i značajno usporava rad algoritma pri svakoj novoj iteraciji. Leaf wise growth podrazumijeva da u svakoj iteraciji pronađemo čvoj koji ima najveću grešku, pa da za dati čvor tražimo novi uslov grananja koji bi potencijalno mogao da smanji datu grešku. Na taj način širimo naše stablo i uvodimo nove uslove grananja samo na mjestima gdje je to potrebno.

### EFB (Exclusive Feature Bundling)

Posljednja optimizacija koju LightGBM uvodi odnosi se na redukciju dimenzionalnosti našeg ulaza. Rekli smo da pri radu sa velikim skupovima podataka pored velikog broja ulaza često imamo i problem da svaki ulaz ima i veliki broj kolona. Kako bismo ubrzali rad algoritma veoma je bitno da nađemo načine da redukujemo broj kolona (featurea). EFB predstavlja tehniku koja se zasniva na ideji da su u praksi često neke kolone sparse vektori, odnosno da imaju veliki broj nula. Ukoliko se u datasetu nalazi više takvih kolona koje nemaju u isto vrijeme ne nula vrijednosti EFB će takve kolone spojiti u jednu kolonu. EFB značajno ubrzava rad i smanjuje upotrebu memorije.

In [1]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split

wine = load_wine()
X = wine.data
y = wine.target

print(X[0])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

[1.423e+01 1.710e+00 2.430e+00 1.560e+01 1.270e+02 2.800e+00 3.060e+00
 2.800e-01 2.290e+00 5.640e+00 1.040e+00 3.920e+00 1.065e+03]


In [2]:
import lightgbm as lgb
from sklearn.metrics import classification_report

# kreiramo klasifikator na bazi lightgbm-a
lgbm_clf = lgb.LGBMClassifier()

# isti interfejs za trening kao i prije
lgbm_clf.fit(X_train, y_train)

# isti interfejs za inferenciju
y_pred = lgbm_clf.predict(X_test)

# classification report funkcija vraca dictionary svih bitnih metrika
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        14
           1       1.00      1.00      1.00        14
           2       1.00      1.00      1.00         8

    accuracy                           1.00        36
   macro avg       1.00      1.00      1.00        36
weighted avg       1.00      1.00      1.00        36



## Pretraga hiperparametara

Većina ozbiljnijih algoritama poput LightGBM-a ima veliki broj parametara koje treba isprobati da bismo vidjeli koja kombinacija daje najbolje performanse na našem skupu podataka. Zbog toga bitno je da imamo jasno definisane korake koji nam mogu dati informacije o tome koja je kombinacija najbolja. Danas ćemo obraditi dva pristupa pretrage hiperparametara.

### Validacioni skup podataka

Prvi pristup podrazumijeva kreiranje novog skupa podataka koji se zove validacioni skup i koji se koristi za validaciju određene kombinacije parametara. Validacioni skup sadrži ulaze koji se ne nalaze ni u trening ni u testnom skupu. Obično se kreira tako što se za testni skup uzme veći procenat ulaza, pa se onda testni skup dodatno podijeli na dva jednaka dijela. Pored pretrage hiperparametara validacioni skup se koristi da spriječi overfitting, odnosno za tehniku ranog zaustavljanja o kojoj smo diskutovali prošli put. Ukoliko imamo validacioni skup algoritam se izvršava sve dok njegove performanse ne počnu da se pogoršavaju na validacionom skupu (pogledati prezentaciju za vizualizaciju).

#### Proces pretrage hiperparametara

Pri radu sa validacionim skupom uvijek imamo isti skup koraka koje koristimo za pretragu hiperparametara:
1. Definišemo moguće varijacije parametara - skup parova (parametar algoritma, lista vrijednosti koje želimo isprobati).
2. Za svaku moguću kombinaciju treniramo algoritam upotrebom trening skupa. Pri svakoj iteraciji provjeravamo kakve su performanse algoritma na validacionom skupu (recimo kakva je neka od metrika precission ili recall). Prekidamo trening kada performanse na validacionom skupu počnu da opadaju.
3. Pamtimo kakve su bile performanse algoritma na validacionom skupu
4. Biramo kombinaciju parameteara koja daje najbolje rezultate
5. Dodatno treniramo model sa najboljim parametrima i testiramo njegove rezultate na testnom skupu kako bismo dobili konačne performanse

#### Zašto novi skup?

Veoma je bitno da se testni skup ne koristi kao validacioni skup. Glavni razlog za to je što želimo da testnim skupom emuliramo rad algoritma u realnom okruženju odnosno da vidimo kako će raditi sa ulazima koje nikad prije nije vidio. Ukoliko koristimo testni skup kao validacioni može se desiti da će algoritam početi u nekoj mjeri da optimizuje u odnosu na testni skup, čime više naši konačni rezultati više ne predstavljaju realnu sliku, jer je vršena optimizacija u odnosu na testni skup.

In [ ]:
import numpy as np
import itertools
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, classification_report
from sklearn.datasets import load_breast_cancer
from sklearn.tree import DecisionTreeClassifier

class ManualParameterGrid:
    def __init__(self, param_grid):

        self.param_grid = param_grid
        self._keys = list(param_grid.keys())
        self._values = [param_grid[k] for k in self._keys]
        total = 1
        for vals in self._values:
            if not hasattr(vals, '__len__'):
                raise ValueError(f"Value for {k!r} is not iterable")
            total *= len(vals)
        self._length = total

    def __len__(self):
        return self._length

    def __iter__(self):
        for combo in itertools.product(*self._values):
            yield dict(zip(self._keys, combo))

    def __getitem__(self, idx):
        if idx < 0:
            idx += self._length
        if not (0 <= idx < self._length):
            raise IndexError(f"Index {idx} out of range")
        sizes = [len(v) for v in self._values]
        combo = {}
        for key, vals, size in zip(self._keys, self._values, sizes):
            idx, rem = divmod(idx, np.prod(sizes[sizes.index(size)+1:]) if size != sizes[-1] else idx)
            combo[key] = vals[idx]
            sizes.pop(0)
        return combo


breast_cancer = load_breast_cancer()
X, y = breast_cancer.data, breast_cancer.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

param_grid = {
    'criterion': ['gini', 'entropy', 'log_loss'],
    'max_depth': [None, 5, 10],
    'min_samples_leaf': [1, 2, 4, 8],
    'min_samples_split': [2, 4, 8],
}

manual_grid = ManualParameterGrid(param_grid)

best_score = -np.inf
best_params = None

for params in manual_grid:
    clf = DecisionTreeClassifier(**params)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_val)
    score = recall_score(y_val, y_pred)
    if score > best_score:
        best_score, best_params = score, params

print("Best score:", best_score)
print("Best parameters:", best_params)

best_clf = DecisionTreeClassifier(**best_params)
best_clf.fit(X_train, y_train)
y_pred = best_clf.predict(X_test)
print(classification_report(y_test, y_pred))

Best score: 1.0
Best parameters: {'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 4, 'min_samples_split': 2}
              precision    recall  f1-score   support

           0       0.96      0.96      0.96        26
           1       0.98      0.98      0.98        60

    accuracy                           0.98        86
   macro avg       0.97      0.97      0.97        86
weighted avg       0.98      0.98      0.98        86



### Unkarsna validacija (Cross Validation)

Glavni problem upotrebe validacionog skupa je što dodatno oduzima podatke iz našeg trening skupa. Ukoliko imamo veliki skup podataka to neće predstavljati problem, međutim često imamo probleme gdje nemamo veliki broj podataka na raspolaganju. U takvim slučajevima svaka trening instanca nam je bitna i oduzimanje od trening skupa može dovesti do situacije da trening skup ne reprezentuje adekvatno realno stanje našeg problema što onda dovodi do lošeg modela. Unakrsa validacija predstavlja drugi pristup validaciji koji ne zahtijeva kreiranje novog skupa podataka. Kod unakrsne validacije trening skup se dijeli u k jednakih dijelova (K folds). Nakon toga za svaku kombinaciju parametara model se trenira sa k-1 dijelova i validira se sa preostalim dijelom. Dati proces se ponavlja za trenugnu kombinaciju hiperparametara sve dok ne izvršimo trening i validaciju za svaku kombinaciju k foldova (odnosno sve dok svaki od foldova ne bude validacioni skup). Konačne performanse za datu kombinaciju parametara dobijamo tako što nađemo srednju vrijednost za svaki fold. U ovom slučaju nemamo gubitak podataka ali imamo povećanu računsku kompelksnost.

In [27]:
from sklearn.base import clone

class ManualGridSearchCV:
    def __init__(self, estimator, param_grid, scoring='neg_mean_squared_error', cv=5, verbose=0, random_state=42):
        if not isinstance(cv, int) or cv < 2:
            raise ValueError("cv must be an integer >= 2")
        self.estimator = estimator
        self.param_grid = ManualParameterGrid(param_grid)
        self.scoring = scoring
        self.cv_folds = cv
        self.verbose = verbose
        self.random_state = random_state
        self._rng = np.random.RandomState(self.random_state)

    def _score(self, y_true, y_pred):
        return self.scoring(y_true, y_pred)

    def _manual_kfold(self, n_samples):
        indices = np.arange(n_samples)
        self._rng.shuffle(indices)
        fold_sizes = np.full(self.cv_folds, n_samples // self.cv_folds, dtype=int)
        fold_sizes[:n_samples % self.cv_folds] += 1

        current = 0
        for fold_size in fold_sizes:
            start, stop = current, current + fold_size
            val_idx = indices[start:stop]
            train_idx = np.concatenate([indices[:start], indices[stop:]])
            yield train_idx, val_idx
            current = stop

    def fit(self, X, y):
        n_samples = X.shape[0]
        best_score = -np.inf
        best_params = None
        self.cv_results_ = []

        for idx, params in enumerate(self.param_grid):
            if self.verbose > 1:
                print(f"[{idx+1}/{len(self.param_grid)}] Testing {params}")
            fold_scores = []

            for train_idx, val_idx in self._manual_kfold(n_samples):
                X_tr, X_val = X[train_idx], X[val_idx]
                y_tr, y_val = y[train_idx], y[val_idx]

                model = clone(self.estimator).set_params(**params)
                model.fit(X_tr, y_tr)
                y_pred = model.predict(X_val)
                fold_scores.append(self._score(y_val, y_pred))

            mean_score = np.mean(fold_scores)
            self.cv_results_.append({'params': params, 'mean_test_score': mean_score})

            if mean_score > best_score:
                best_score, best_params = mean_score, params

        self.best_score_ = best_score
        self.best_params_ = best_params
        self.best_estimator_ = clone(self.estimator).set_params(**best_params)
        self.best_estimator_.fit(X, y)
        return self

    def score(self, X, y):
        return self.best_estimator_.score(X, y)

In [28]:
import numpy as np
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

data = load_wine()
X, y = data.data, data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Kreiramo regresor
reg = DecisionTreeRegressor()

param_grid = {
    'criterion': ['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
    'max_depth': [None, 5, 10],
    'min_samples_leaf': [1, 2, 4, 8],
    'min_samples_split': [2, 4, 8],
}

# Inicijalizujemo objekat za pretragu hiperparametara upotrebom unakrsne validacije
grid_search = ManualGridSearchCV(
    reg,
    param_grid,
    scoring=lambda y_true, y_pred: -mean_squared_error(y_true, y_pred),
    cv=5,
    verbose=2
)

# Fitujemo grid - pretrazujemo prostor hiperparametara
grid_search.fit(X_train, y_train)

print("Best parameters found: ", grid_search.best_params_)
print("Best MSE found: ", grid_search.best_score_ * -1)

# Evaluiramo najbolji model na testnom skupu
best_model = grid_search.best_estimator_
test_prec = best_model.score(X_test, y_test)
print(test_prec)

[1/144] Testing {'criterion': 'squared_error', 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2}
[2/144] Testing {'criterion': 'squared_error', 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 4}
[3/144] Testing {'criterion': 'squared_error', 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 8}
[4/144] Testing {'criterion': 'squared_error', 'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 2}
[5/144] Testing {'criterion': 'squared_error', 'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 4}
[6/144] Testing {'criterion': 'squared_error', 'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 8}
[7/144] Testing {'criterion': 'squared_error', 'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 2}
[8/144] Testing {'criterion': 'squared_error', 'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 4}
[9/144] Testing {'criterion': 'squared_error', 'max_depth': None, 'min_samples_leaf': 4, 'min_sa

# Kompromisi

Prilikom pretrage hiperparametara treba biti veoma pažljiv sa odabirom vrijednosti. Idealno bi bilo da se pretraži što veći skup vrijednosti međutim to značajno povećava vrijeme izvršavanja pa samim tim i račun za mašine koje to moraju da izvrše. Kako bi se rješio taj problem potrebno je identifikovati najbitnije parametre i za svaki od njih pronaći odgovarajući skup vrijednosti. Za dati izbor nema previše pravila i običnno podrazumijeva upotrebu iskustva i Googlanja kako bi se identifikovale bitne vrijednosti. Takođe obično je dobro početi sa velikim razmacima unutar vrijednosti za pretragu, pa zatim smanjivati dati opseg na manje intervale u zavisnosti od detektovanih performansi. 